In [48]:
import assessmentModule as amod
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.signal as ss
%matplotlib qt
"""
For a subset of running LT assessments, look at relationships
between SmO2, tHb, and HR using change in slope.

"""

__author__ = 'Todd Minehardt'


# Define working directory where Matlab files are and output directory.
data_dir = '/Users/todd/data/WARMUP/RUN'
out_dir = '/Users/todd/data/WARMUP/RUN/derivatives'

# Go to working directory.
os.chdir(data_dir)

# Get list of LT assessment IDs from list.
aid_file = '/Users/todd/code/AnalysisTools/config/subset_runs_score_16-18.txt'
with open(aid_file, 'r') as fn:
    aid_list = []
    for line in fn:
        aid_list.append(line)
assessment_ids = [i.strip('\n') for i in aid_list]

# Build a dict keyed on assessment ID, containing all information
# from relevant Matlab files. This returns a dict of dicts.
data = {}
for assess_id in assessment_ids:
    data[assess_id] = amod.getMatlabData(assess_id + '_process.mat')
    data[assess_id].update(amod.getMatlabData(assess_id + '_sweep.mat'))

# Filter data with Savitsky-Golay.
def returnSGFilteredData(x, window_length, polyorder, deriv):
    """Given x, window size, and polynomial order, return
    Savitsky-Golay filtered data.
    """
    return ss.savgol_filter(x.flatten(), 
                            window_length=window_length,
                            polyorder=polyorder,
                            deriv=deriv)

def smooth(x, n):
    """Return moving average for period n."""
    w = np.ones((n, )) / n
    return np.convolve(x, w, mode='valid')

In [104]:
%matplotlib inline

In [105]:
# Define start and end indices for data arrays.
start_idx = 161
end_idx = None
to_show = ['56470b06adac18e1278b4586',
              '564a1f52adac183f568b456e',
              '564e180b07c51979098b4573',
              '566047c6adac18097d8b4568',
              '566d4502adac1812528b4569',
              '5676ca9207c51951518b4568',
              '5678384c07c519b85b8b456a',
              '568006b8adac1834758b456a',
              '568aab2fadac183f618b4568']
# Plot remaining assessments.
for assess_id in data.keys():
#for assess_id in to_show:
    
    if 'HR' in data[assess_id]['process'].keys():
        HR = data[assess_id]['process']['HR']
    elif 'HR' in data[assess_id]['sweep'].keys():
        HR = data[assess_id]['sweep']['HR']
    
    if 'SmO2' in data[assess_id]['process'].keys():
        SmO2 = data[assess_id]['process']['SmO2']
    elif 'SmO2' in data[assess_id]['sweep'].keys():
        SmO2 = data[assess_id]['sweep']['SmO2']
        
    if 'tHb' in data[assess_id]['process'].keys():     
        tHb = data[assess_id]['process']['tHb']
    
    # Truncate arrays on start and end indices.
    HR = HR[start_idx:end_idx]
    SmO2 = SmO2[start_idx:end_idx]
    tHb = tHb[start_idx:end_idx]
    
    # Three subplots sharing x axis
    f, (ax1, ax2, ax3) = plt.subplots(3, sharex=True, sharey=False)
    
    # Plot the polynomial fit for HR.
    idx = np.isfinite(HR)
    y = HR[idx]
    x = np.arange(len(y))
    z = np.polyfit(x, y, deg=4)
    fit = np.polyval(z, np.arange(len(HR)))
    
    x_time_ticks = np.arange(len(fit)) / 5
    ax1.plot(x_time_ticks, 
             fit, 
             color='red',
             label='HR fit')
        
    ax1.plot(x_time_ticks, 
             HR, 
             color='red',
             label='HR')

    ax1.set_ylim(np.nanmin(HR), 
                 np.nanmax(HR))
    ax1.set_ylabel('HR (bpm)',
                   color='red')
    for tl in ax1.get_yticklabels():
        tl.set_color('red')   
    ax1.set_title('Assessment ' + assess_id)
    
    ax1.minorticks_on()
    ax1.xaxis.grid(True, which='major')
    ax1.xaxis.grid(True, which='minor')
    ax1.yaxis.grid(True, which='major')
    ax1.yaxis.grid(False, which='minor')
    ax1.tick_params(axis='y', which='minor', left='off')
    #ax1.set_xlim(0, 3600)
    
    # Plot the polynomial fit for SmO2.
    idx = np.isfinite(SmO2)
    y = SmO2[idx]
    x = np.arange(len(y))
    z = np.polyfit(x, y, deg=4)
    fit = np.polyval(z, np.arange(len(SmO2)))
    x_time_ticks = np.arange(len(fit)) / 5
    
    ax2.plot(x_time_ticks,
             fit,
             color='green',
             label='SmO2 fit')
    ax2.plot(x_time_ticks,
             SmO2,
             color='green',
             label='SmO2',
             alpha=0.25)
    ax2.set_ylim(np.nanmin(SmO2), 
                 np.nanmax(SmO2))
    ax2.set_ylabel('SmO2 (%)',
                   color='green')
    for tl in ax2.get_yticklabels():
        tl.set_color('green')
        
    ax2.minorticks_on()
    ax2.xaxis.grid(True, which='major')
    ax2.xaxis.grid(True, which='minor')
    ax2.yaxis.grid(True, which='major')
    ax2.yaxis.grid(False, which='minor')
    ax2.tick_params(axis='y', which='minor', left='off')
    #ax2.set_xlim(0, 3600)
    
    # Find local maximum from end of stage 1 to stage 8.
    local_max = start_idx + np.argmax(fit[900:7200]) / 5
    ax2.axvline(local_max, 
                color='green',
                linewidth=3,
                label=local_max)
    ax2.legend()
        
    # Plot the polynomial fit for tHb.
    idx = np.isfinite(tHb)
    y = tHb[idx]
    x = np.arange(len(y))
    z = np.polyfit(x, y, deg=4)
    fit = np.polyval(z, np.arange(len(tHb)))
    
    x_time_ticks = np.arange(len(fit)) / 5
    ax3.plot(x_time_ticks,
             fit,
             color='blue',
             label='tHb fit')
    ax3.plot(x_time_ticks,
             tHb,
             color='blue',
             label='tHb',
             alpha=0.25)
    ax3.set_ylim(np.nanmin(tHb),
                 np.nanmax(tHb))
    ax3.set_ylabel('tHb fit (g/dL)',
                   color='blue')

    for tl in ax3.get_yticklabels():
        tl.set_color('blue')
    ax3.set_xlabel('time (s)')
    
    ax3.minorticks_on()
    ax3.xaxis.grid(True, which='major')
    ax3.xaxis.grid(True, which='minor')
    ax3.yaxis.grid(True, which='major')
    ax3.yaxis.grid(False, which='minor')
    ax3.tick_params(axis='y', which='minor', left='off')
    #ax3.set_xlim(0, 3600)
    
    # Find local maximum from end of stage 1 to stage 8.
    local_max = start_idx + np.argmax(fit[900:7200]) / 5
    ax3.axvline(local_max, 
                color='blue',
                linewidth=3,
                label=local_max)
    ax3.legend()

    
    # Fine-tune figure; make subplots close to each other and hide x ticks for
    # all but bottom plot.
    f.subplots_adjust(hspace=0)
    plt.setp([a.get_xticklabels() for a in f.axes[:-1]], visible=False)

    # Plot parameters.
    plt.rcParams['figure.figsize'] = 10, 8
    plt.rcParams['legend.loc'] = 'best'
    out_file = os.path.join(out_dir, assess_id + '.png')
    plt.savefig(out_file)
    plt.close()
    #plt.show()

In [57]:
exercise_times = []
for aid in assessment_ids:
    time = int(data[aid]['sweep']['time'][-1])
    exercise_times.append([aid, time])


In [58]:
len(exercise_times)

91

In [109]:
users = []
for i in assessment_ids:
    uid = data[i]['assessment']['user_id']
    if uid.shape[0] == 0:
        uid = 99999
    elif uid.shape[0] == 1:
        uid = np.asscalar(uid)
    print(uid)
    users.append(uid)

491
2238
154
1041
154
664
870
523
710
125
2282
2452
2214
2385
2059
543
2214
543
814
2378
2368
2458
1015
2547
2385
374
2442
43
646
408
2445
834
2506
2522
2506
2293
2575
567
554
2558
2672
2690
1015
2482
2366
2072
1808
2693
2654
2293
2647
1999
2738
2647
2766
342
611
2674
2732
2735
237
2629
680
2673
2408
1785
2758
2886
411
2929
2068
2841
2789
1256
2322
333
2878
2967
2912
2680
2895
688
1630
1306
2979
2960
3001
2991
2973
2973
3035


In [73]:
np.asscalar(data['56470b06adac18e1278b4586']['assessment']['stage_at_lt_value'])

2.324608

In [89]:
data['56470b06adac18e1278b4586']['assessment'].keys()

dict_keys(['completed_on', 'lthr', 'allow_download', 'completed_stages', 'calculated_lt1_speed', 'protocol', 'days_per_week', 'alpha__id', 'calculated_lt_speed', 'created_at', 'entry_type', 'months_training', 'links', 'sport', 'training_zones', 'status', 'stage_at_lt', 'duration', 'worker_details', 'user_id', 'stage_at_lt_value', 'distance_per_week', 'lt1hr'])

In [114]:
sorted(users)

['1015',
 '1015',
 '1041',
 '125',
 '1256',
 '1306',
 '154',
 '154',
 '1630',
 '1785',
 '1808',
 '1999',
 '2059',
 '2068',
 '2072',
 '2214',
 '2214',
 '2238',
 '2282',
 '2293',
 '2293',
 '2322',
 '2366',
 '2368',
 '237',
 '2378',
 '2385',
 '2385',
 '2408',
 '2442',
 '2445',
 '2452',
 '2458',
 '2482',
 '2506',
 '2506',
 '2522',
 '2547',
 '2558',
 '2575',
 '2629',
 '2647',
 '2647',
 '2654',
 '2672',
 '2673',
 '2674',
 '2680',
 '2690',
 '2693',
 '2732',
 '2735',
 '2738',
 '2758',
 '2766',
 '2789',
 '2841',
 '2878',
 '2886',
 '2895',
 '2912',
 '2929',
 '2960',
 '2967',
 '2973',
 '2973',
 '2979',
 '2991',
 '3001',
 '3035',
 '333',
 '342',
 '374',
 '408',
 '411',
 '43',
 '491',
 '523',
 '543',
 '543',
 '554',
 '567',
 '611',
 '646',
 '664',
 '680',
 '688',
 '710',
 '814',
 '834',
 '870']